# Calulate Dealy Time of Applying Magnetic Field
**Author**: `Kuan-Chia Chiu`  
**Update**: `2018-12-12`  
**Package**: `pyvisa`, `serial`, `ipywidgets`(*optional*), `plotly`(*optional*)  
**Driver**: `NI-VISA`

In [1]:
import visa
import time
import serial
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import datetime
import time


rm = visa.ResourceManager()
res = rm.list_resources()
vna_dict =[]
for x in range(0, len(res)):
    vna_dict.append(res[x])
    print("{}. ".format(x) + res[x] + ", \n")
print("\n")

0. ASRL4::INSTR, 

1. ASRL5::INSTR, 

2. ASRL6::INSTR, 

3. GPIB0::6::INSTR, 





In [2]:
KEPCO_address = widgets.ToggleButtons(
    options=vna_dict,
    description="Power Sup.",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=[],
#     icons=['check'] * 3
)

In [3]:
LakeShore_address = widgets.Text()
accordion = widgets.Accordion(children=[KEPCO_address, LakeShore_address])
accordion.set_title(0, 'Power Supply (KEPCO)')
accordion.set_title(1, 'Gauss meter (LakeShore)')
accordion

Accordion(children=(ToggleButtons(button_style='success', description='Power Sup.', options=('ASRL4::INSTR', '…

In [5]:
ser = serial.Serial(LakeShore_address.value, 57600,bytesize=7, parity=serial.PARITY_ODD, stopbits=1)  # Lake Shore 425
LakeShore_address.value

'COM6'

In [21]:
sweep_current = 40
sweep_pts = 1601
KEPCO = rm.open_resource(KEPCO_address.value)

KEPCO.timeout=10000
KEPCO.write("FUNC:MODE CURR")
KEPCO.write("volt 0.000;curr 0.000")
KEPCO.write("volt  20.000;curr {:.4f}".format(sweep_current/(sweep_pts-1)))

Field_data = []
Time = []
origin = datetime.now()
KEPCO.write("OUTPUT ON")
for i in range(200):
    time.sleep(0.005)
    current_time = datetime.now()
    ser.write(b'RDGFIELD?\n')
    Field = ser.readline()
    Field_data.append(Field.decode('utf8'))
    Time.append(current_time.second + (current_time.microsecond-origin.microsecond)/1000000 - origin.second)
    
KEPCO.write("OUTPUT OFF")
KEPCO.write("volt 0.000;curr 0.000")
#ser.close()


(23, <StatusCode.success: 0>)

# Plot Data

In [22]:
import plotly as py
from plotly.graph_objs import Scatter, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
trace = go.Scatter(x=Time, y=Field_data, name='Original data')
iplot([trace], filename='test')

## Conclusion
When the max step current (having been checked around 40/1601 A) is applied on electromagnet, there is a delay to reach a stable field.  
Thus, we check the figure above.  
We need to wait at least **1s** to make it stable.  